In [1]:
from GeoBases import GeoBase

In [2]:
geo_c = GeoBase(data='ori_por', verbose=False)

In [3]:
!cat stations.txt | head -n 10

ACW00011604  17.1167  -61.7833   10.1    ST JOHNS COOLIDGE FLD                       
ACW00011647  17.1333  -61.7833   19.2    ST JOHNS                                    
AE000041196  25.3330   55.5170   34.0    SHARJAH INTER. AIRP            GSN     41196
AEM00041194  25.2550   55.3640   10.4    DUBAI INTL                             41194
AEM00041217  24.4330   54.6510   26.8    ABU DHABI INTL                         41217
AEM00041218  24.2620   55.6090  264.9    AL AIN INTL                            41218
AF000040930  35.3170   69.0170 3366.0    NORTH-SALANG                   GSN     40930
AFM00040938  34.2100   62.2280  977.2    HERAT                                  40938
AFM00040948  34.5660   69.2120 1791.3    KABUL INTL                             40948
AFM00040990  31.5000   65.8500 1010.0    KANDAHAR AIRPORT                       40990
cat: write error: Broken pipe


In [4]:
import pandas as pd

In [5]:
stationstxt = ""
with open("stations.txt") as input:
    stationstxt = input.read()

In [6]:
#Extract the data from file
stations = stationstxt.split("\n")

#Remove last line
stations = stations[:-1]

stations = map(lambda line: [line[0:11],float(line[13:20]),float(line[22:30]),line[41:71]], stations)

In [7]:
len(stations)

100100

In [8]:
stations[:5]

[['ACW00011604', 17.1167, -61.7833, 'ST JOHNS COOLIDGE FLD         '],
 ['ACW00011647', 17.1333, -61.7833, 'ST JOHNS                      '],
 ['AE000041196', 25.333, 55.517, 'SHARJAH INTER. AIRP           '],
 ['AEM00041194', 25.255, 55.364, 'DUBAI INTL                    '],
 ['AEM00041217', 24.433, 54.651, 'ABU DHABI INTL                ']]

In [9]:
def getCity(lat,lng):
    try:
        l = [[k,int("0"+geo_c.get(k)["population"])] for _,k in geo_c.findNearPoint((lat, lng), 10)]
        l = sorted(l, key=lambda city: city[1], reverse=True)[0]
        code = l[0]
        return geo_c.get(code,'city_code')
    except:
        return ""

stations = map(lambda s: s + [getCity(s[1],s[2])],stations)

In [10]:
stationsdf = pd.DataFrame(stations)
stationsdf.columns = ["id","lat","lng","name","asciicity"]

In [11]:
citywithweather = stationsdf[stationsdf["asciicity"]!=""]

In [12]:
len(citywithweather)

13333

In [13]:
grcity = citywithweather.groupby("asciicity").first().reset_index()

In [14]:
len(grcity)

3450

In [15]:
open("citiesweather.csv","w").write(grcity.to_csv())

In [18]:
grcity[grcity["asciicity"].str.contains("MAD")]

,asciicity,id,lat,lng,name
1612,MAD,SP000003195,40.4117,-3.6781,MADRID - RETIRO


In [19]:
grcity.sample(5)

,asciicity,id,lat,lng,name
2058,PCU,US1MSPR0002,30.5240,-89.6360,PICAYUNE 2.5 E
3226,YIB,CA006020379,48.7500,-91.6167,ATIKOKAN
2139,PNY,IN022043400,11.9200,79.8300,PONDICHERY OBSY
207,BBD,US1TXMCC007,31.1186,-99.3427,BRADY 2.2 ESE
1131,HVN,US1CTNH0003,41.3500,-72.9391,HAMDEN 3.8 SW
